In [ ]:
20

In [ ]:
20

In [ ]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model




### Register model

In [ ]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [ ]:


from federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_with_summation_1MINE_VAE import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

### Register metric

### Create the config

#### Set data, model, trainer and metric

In [ ]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("scripts/B-FHTL_exp_scripts/Graph-DT/fed_dom_sep_MI.yaml")
cfg.data.save_dir = 'test_dir'
# cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
cfg.model.dropout = 0.
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']
cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance= 1e2
cfg.params.diff_importance = 1
cfg.params.mine_lr = 0.01
cfg.params.eps=1e-20
cfg.params.p=0.
cfg.params.lam = 0.01
cfg.params.kld_importance = 0.


#### configure other options

In [ ]:
#cfg.use_gpu = True 0.05 [0.02-0.03] 0.04-0.06
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'

cfg.federate.method = \
    f'Laplacian_MINE_VAE_test_no_KLD'

#cfg.federate.local_update_steps = 20000000
cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 2000
cfg.federate.client_num = 2
cfg.early_stop.patience = 3000
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 2.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'
cfg.eval.count_flops = False


In [ ]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(F/home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competitionalse)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [ ]:
from yacs.config import CfgNode
client_cfg_file = "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_theirs.yaml"# "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [ ]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


In [ ]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_MI_client import LaplacianDomainSeparationMIClient
from federatedscope.contrib.workers.laplacian_server_dom_sep import LaplacianServerDomSep

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


In [ ]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_1MINE_VAE_client import LaplacianDomainSeparation1MINEClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianDomainSeparation1MINEClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

client 0 times: 0.346s
bei normal fedavg: 0.15s
mit MINE: 1.7 Sekunden


+

In [ ]:
777-431


In [ ]:
data

In [ ]:
import os

,,0After 10 rounds:

diff: 0, csd: 0:
server: -8
c1 t/val: 0.375 /0.37
c2 t/val: 0.678 /0.606
c3 t/val: 0.699 /0.608
c4 t/val: 0.599 /0.537

diff: 0, csd: 0:
server: -4.5
c1 t/val: 0.372879 /0.328
c2 t/val: 0.678992 /0.605809
c3 t/val: 0.70166 /0.608609
c4 t/val: 0.597455 /0.534253



diff: 10, csd: 0:
server: -35
c1 t/val: 0.482/0.52
c2 t/val: 0.694/0.65
c3 t/val: 0.724/0.65
c4 t/val: 0.674/0.575

diff: 1, csd: 0:
server: -9.6
c1 t/val: 0.378194/0.319349
c2 t/val: 0.67461/0.609163
c3 t/val: 0.705559/0.615019
c4 t/val: 0.605581/0.572086

diff: 0, csd: 1e4:
server: -18
c1 t/val: 0.280962/0.23074
c2 t/val: 0.682652/0.603175
c3 t/val: 0.699094/0.67027
c4 t/val: 0.594985/0.609611

diff: 0 csd: 1e8
server: -18
c1 t/val: 0.2606/0.348(0.197)
c2 t/val: 0.653/0.0.602
c3 t/val: 0.691/0.597
c4 t/val: 0.581/0.629

diff: 0 csd: 1e8
server: -12.7
c1 t/val: 0.262647/0.249875
c2 t/val: 0.649273/0.0.600435
c3 t/val: 0.696132/0.615158
c4 t/val: 0.586597/0.656992


diff: 0.1 csd: 0
server: -11
c1 t/val: 0.2606/0.332837
c2 t/val: 0.678326/0.0.610099
c3 t/val: 0.704083/0.616179
c4 t/val: 0.604177/0.559658




In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']